# Lab Report #2

## Problem Statement

Our goal is to manipulate specific variables in order to simulate four orbital scenarios:
1. **Circular Run** - representing Earth’s nearly circular orbit around the Sun.
2. **Two Elliptical Runs** - showing systems where orbits take on oval shapes.
3. **Binary System Run** - simulating a system where the Earth’s mass is artificially scaled to match the Sun’s mass. In this case, both objects orbit a common center of mass rather than the Earth orbiting the Sun alone.
   
For each scenario, we also generated four graphs:
* Earth & Sun momentum vs. time.
* Total system momentum vs. time.
* Earth & Sun velocities vs. time.
* A custom graph comparing Earth & Sun positions vs. time.

## Physics Relevant to the Problem
The simulation is grounded in the Momentum Principle, which relates force, momentum, and velocity. The momentum of each body is given by:
* $p_{Earth} \times = m_{Earth} \times v_{Earth}$
* $p_{Sun}=m_{Sun} \times v_{Sun}$
We require the following physical parameters:
* $m_{Earth}=5.972 \times 10^{24}kg$
* $m_{Sun}=1.989 \times 10^{30}kg$
* $R_{Earth}=6.4 \times 10^6m$
* $R_{Sun}=7.0 \times 10^8m$
* $G = 6.7 \times 10^{−11} \frac {N \times m^2} {kg^2}$

The Earth begins at position  ($1.49 \times 10^{11}$,0,0) m  with an initial velocity  $v_y=2.978×10^4$ m/s, while the Sun starts at the origin with zero velocity. The simulation time step is one hour ($dt=3600$ s), and the total run length is one simulated year.

The gravitational force between the two objects is modeled by: $F= -\frac{G \times m_{Earth} \times m_{Sun}}{|r|^2} \times \hat{r}$

This force updates each object’s momentum and velocity at every timestep: (1) $p_f=p_i+FΔt$, (2) $v=\frac{p}{m}$, and (3) $r_f=r_i+vΔt$

This iterative approach reflects Newton’s Method of numerical integration, allowing us to update velocity, position, and momentum continuously throughout the simulation.

##  How the Code Solves Our Problem
The code performs three main tasks:
1. Initialize Objects – Create the Sun (orange sphere at origin) and Earth (blue sphere offset in x). Their masses, radii, and initial velocities are set here.
2. Initialize Momentum and Constants – Calculates initial momenta, stores baseline values, and defines the gravitational constant.
3. Run Simulation Loop – The program iterates over time until Earth makes one revolution around the Sun:

    * Compute distance vector $r$
    * Calculate gravitational force
    * Update Earth and Sun momentum using $Δp=FΔt$. Note the opposite signs due to Newton’s Third Law.
    * Recompute velocities and update positions.
    * Record data for graphing.
    * Check orbital completion by comparing momentum direction.
    * Update reference values for the next iteration.

Through this loop, the simulation produces both the visual orbits and the associated graphs of momentum, velocity, and position.

## The Code

In [ ]:
from vpython import *


def create_earth(position = vec(1.49e11, 0, 0), mass = 5.972e24, velocity = vec(0, 1.0 * 2.978e4, 0)):
    """
    Create a VPython sphere object representing the Earth.
    
    Args:
        pos (vpython.vec):
            Initial position of Earth
        mass (float):
            Mass of Earth in kg
        velocity (vpython.vec):
            Initial velocity vector of Earth

    Returns:
        A VPython sphere object with additional physical attributes.
    """
    earth = sphere(
        pos = position,         # Earth-Sun distance,
        mass = mass,            # Earth mass (kg)
        radius = 6 * 6.4e8,     # Scaled Earth radius (m) for visibility
        v = velocity,           # Initial orbital velocity (m/s)
        color = color.blue,
        make_trail = True
    )

    return earth


def create_sun(position=vec(0, 0, 0), mass=1.989e30, velocity=vec(0,0,0)):
    """
    Create a VPython sphere object representing the Sun.
    
    Args:
        pos (vpython.vec):
            Initial position of Sun (default = (0, 0, 0)).
        mass (float):
            Mass of Sun in kg (default = 1.989e30 kg).
        velocity (vpython.vec):
            Initial velocity vector of Sun (default is 0 along all axes).

    Returns:
        A VPython sphere object with additional physical attributes.
    """
    
    sun = sphere(
        pos = position,          # Centers Sun in our simulation
        mass = mass,             # Sun mass (kg)
        radius = 10 * 7e8,       # Scaled Sun radius (m) for visibility
        v = velocity,            # Initial orbital velocity (m/s)
        color = color.orange,
        make_trail = True
    )

    return sun


def run_simulation(earth=None, sun=None):
    """
    Run a simplified Sun–Earth orbital simulation and optionally display 
    VPython graphs of work and kinetic energy over time.

    Args:
            
        Earth (vpython.sphere):
            If not None, uses the Earth object passed in by user.
            
        Sun (vpython.sphere):
            If not None, uses the Sun object passed in by user.
    """

    # 3D simulation requires vpython.sphere objects
    # Uses the Earth object passed by user, otherwise uses "real-world" properties for Earth
    if earth is None:
        earth = create_earth()

    # Uses the Sun object passed by user, otherwise uses "real-world" properties for Sun
    if sun is None:
        sun = create_sun()

    # Initial momenta
    earth.p = earth.mass * earth.v
    sun.p = sun.mass * sun.v
    earth.old_p = earth.p 
    
    # Universal gravitational constant
    G = 6.7e-11
    
    # Time variables
    t = 0               # Initial simulation time
    dt = 60 * 60        # Represents timestep of 1 hour
    earth.T = 0         # Counter for orbital period
    myrate = 1500       # Controls animation speed

    # Momentum graph (Earth & Sun components)
    gr_p = graph(title=' Earth & Sun Momentum Diagram ', xtitle =' time (s)', ytitle=' p (kg⋅m/s)')
    p_plot_E_x = gcurve(graph=gr_p, color=color.blue )
    p_plot_E_y = gcurve(graph=gr_p, color=color.green )
    p_plot_E_z = gcurve(graph=gr_p, color=color.cyan )
    p_plot_S_x = gcurve(graph=gr_p, color=color.red )
    p_plot_S_y = gcurve(graph=gr_p, color=color.orange )
    p_plot_S_z = gcurve(graph=gr_p, color=color.yellow )
    
    # Total momentum graph (system momentum check)
    gr_pt = graph(title=' Total Momentum Diagram ', xtitle=' time (s) ', ytitle=' p (kg⋅m/s)')
    p_plot_t_x = gcurve(graph=gr_pt, color=color.red )
    p_plot_t_y = gcurve(graph=gr_pt, color=color.blue )
    p_plot_t_z = gcurve(graph=gr_pt, color=color.green )
    
    # Velocity graph (Earth & Sun components)
    gr_v = graph(title=' Earth & Sun Velocity Diagram ', xtitle=' time (s) ', ytitle=' v (m/s)')
    v_plot_E_x = gcurve(graph=gr_v, color=color.blue )
    v_plot_E_y = gcurve(graph=gr_v, color=color.green )
    v_plot_E_z = gcurve(graph=gr_v, color=color.cyan )
    v_plot_S_x = gcurve(graph=gr_v, color=color.red )
    v_plot_S_y = gcurve(graph=gr_v, color=color.orange )
    v_plot_S_z = gcurve(graph=gr_v, color=color.yellow )
    
    # Position graph (Earth & Sun components)
    gr_position = graph(title=' Earth & Sun position Diagram ', xtitle=' time (s) ', ytitle=' pos (m) ')
    position_plot_E_x = gcurve(graph=gr_position, color=color.blue )
    position_plot_E_y = gcurve(graph=gr_position, color=color.green )
    position_plot_E_z = gcurve(graph=gr_position, color=color.cyan )
    position_plot_S_x = gcurve(graph=gr_position, color=color.red )
    position_plot_S_y = gcurve(graph=gr_position, color=color.orange )
    position_plot_S_z = gcurve(graph=gr_position, color=color.yellow )

    years = []
    
    ONE_YEAR = 60 * 60 * 24 * 365

    # Simulation loop to update variables over time
    while t < ONE_YEAR:  # Runs simulation for one in-simulation year
        rate(myrate)
        
        # Relative position vector from Earth to Sun
        r = earth.pos - sun.pos
        
        # Gravitational force on Earth due to Sun (Newton’s law of gravitation)
        F = - G * earth.mass * sun.mass / r.mag**2 * r.hat 
        
        # Update momenta
        earth.p = earth.p + F * dt
        sun.p = sun.p - F * dt
        
        # Update velocities
        earth.v = earth.p / earth.mass
        sun.v = sun.p / sun.mass 
        
        # Update positions
        earth.pos = earth.pos + earth.v * dt
        sun.pos = sun.pos + sun.v * dt
    
        # Graphs momentum components if required
        p_plot_E_x.plot(pos=(t,earth.p.x))
        p_plot_E_y.plot(pos=(t,earth.p.y))
        p_plot_E_z.plot(pos=(t,earth.p.z))
        p_plot_S_x.plot(pos=(t,sun.p.x))
        p_plot_S_y.plot(pos=(t,sun.p.y))
        p_plot_S_z.plot(pos=(t,sun.p.z))
        
        # Graphs total system momentum if required
        p_plot_t_x.plot(pos=(t,sun.p.x+earth.p.x))
        p_plot_t_y.plot(pos=(t,sun.p.y+earth.p.y))
        p_plot_t_z.plot(pos=(t,sun.p.z+earth.p.z))
        
        # Graphs velocity components if required
        v_plot_E_x.plot(pos=(t,earth.v.x))
        v_plot_E_y.plot(pos=(t,earth.v.y))
        v_plot_E_z.plot(pos=(t,earth.v.z))
        v_plot_S_x.plot(pos=(t,sun.v.x))
        v_plot_S_y.plot(pos=(t,sun.v.y))
        v_plot_S_z.plot(pos=(t,sun.v.z))
        
        # Graphs position components if required
        position_plot_E_x.plot(pos=(t,earth.pos.x))
        position_plot_E_y.plot(pos=(t,earth.pos.y))
        position_plot_E_z.plot(pos=(t,earth.pos.z))
        position_plot_S_x.plot(pos=(t,sun.pos.x))
        position_plot_S_y.plot(pos=(t,sun.pos.y))
        position_plot_S_z.plot(pos=(t,sun.pos.z))
        
        # Increment time
        t += dt

        # Detect when Earth completes one orbit:
        # Check when momentum in x-direction flips sign (crossing orbit point)
        if earth.old_p.x > 0 and earth.p.x < 0 :
            simulated_days = round(earth.T / (60 * 60 * 24), 2)

            # Correctly calculates year based on the number of days passed
            if len(years) == 0:
                years.append(simulated_days)
            else:
                last_revolution = years[-1]
                revolution_days = round(simulated_days - last_revolution, 2)
                years.append(simulated_days)
                print(f"Year length: {revolution_days} days")
        
        # Store current momentum for next loop check
        earth.old_p = earth.p
        earth.T = earth.T + dt
        

## Simulation #1 - Circular Orbit
![Simulation](../assets/gifs/Lab-2/Earth_Orbiting_Sun_1.gif)

**Observations**: Running the simulation with the default variables produces a circular orbit of the Earth around the Sun. The Earth’s momentum changes continuously as it orbits, while the Sun’s momentum also changes but at a much smaller rate due to its vastly greater mass. Visually, the Sun appears stationary, but the graphs confirm that it moves slightly in response to Earth’s gravitational pull. This matches expectations for a circular Earth–Sun system. However, the orbital period printed out is slightly off: the simulation yields ~358 days per orbit instead of the real-world 365. This discrepancy arises because our model assumes a perfectly circular orbit, while Earth’s actual orbit is elliptical. Despite the difference, the output is reasonable: it reproduces the correct long-term orbital behavior within the limits of the simplified model.

### Graphs

#### Graph #1 – Earth and Sun Momentum
![Momentum Graph](../assets/img/Lab-2/Run-1/2-1-Momentum.png)

* Sun’s momentum (red, orange, yellow = x, y, z):
    * In the x-direction, the Sun oscillates between positive and negative momentum as the Earth tugs on it from alternating sides of the orbit. This mirrors Earth’s momentum in the x-direction, but in opposite sign, consistent with Newton’s Third Law.
    * In the y-direction, the Sun’s momentum cycles upward and downward, pausing at 0 when Earth completes an orbit. Again, this mirrors Earth’s y-momentum but offset.
    * In the z-direction, both Earth and Sun remain at 0 since there is no motion toward or away from the observer.
* Earth’s momentum (blue, green, cyan = x, y, z):
    * The x-component alternates between positive and negative values, showing Earth’s left–right motion across each orbit.
    * The y-component starts positive (upward), then decreases into the negative (downward) direction before repeating, corresponding to Earth’s vertical orbital path.
    * The z-component remains 0, as expected.

Together, the graphs show Earth and Sun momenta oscillating in equal and opposite ways, confirming conservation of momentum.

#### Graph #2 – Total Momentum p(t)
![Total Momentum Graph](../assets/img/Lab-2/Run-1/2-1-Total-Momentum.png)

This graph sums the Earth and Sun momenta in each direction (red = x, blue = y, green = z
* The z-component remains constant at 0, consistent with no motion toward/away from the observer.
* The y-component is constant at about $1.8×10^{29} kg \times m/s$. This occurs because the Earth’s decreasing momentum exactly balances the Sun’s increasing momentum in the y-direction.
* The x-component also cancels out, since Earth and Sun’s x-momenta are mirrored about 0.

Thus, the system’s total momentum is conserved in all directions, as expected.

#### Graph #3 – Earth and Sun Velocity
![Velocity Graph](../assets/img/Lab-2/Run-1/2-1-Velocity.png)

* Sun’s velocity is not visible because its enormous mass makes its velocity extremely small (less than 1 m/s). This is consistent with its small momentum compared to its mass.
* Earth’s velocity closely follows its momentum curve, since $v=\frac{p}{m}$. Peaks and troughs in velocity align directly with the momentum graph.

#### Graph #4 – Earth and Sun Position
![Position Graph](../assets/img/Lab-2/Run-1/2-1-Position.png)

* The Earth’s position curve is the integral of its velocity graph, showing how orbital position evolves over time.
    * In the x-direction, Earth starts far to the right, moves leftward as velocity becomes negative, then reverses when velocity becomes positive again.
    * In the y-direction, Earth starts upward, then transitions downward as its velocity switches from positive to negative.
* The Sun’s position shows almost no change because its velocity is negligible.
* As expected, the z-components remain zero throughout.

Overall, the position and velocity graphs are consistent: velocity determines the slope of the position curve, and changes in direction align perfectly between the two.

## Simulation #2 – Elliptical Orbit
![Simulation](../assets/gifs/Lab-2/Earth_Orbiting_Sun_2.gif)

### Overview
**Setup**: For this run, the Earth’s initial y-velocity was modified to be $1.3 \times 2.978e4$ m/s. This adjustment causes the Earth to deviate from a circular orbit and instead follow an elliptical trajectory. The goal of this simulation is to observe the orbital shape, track changes in momentum and velocity of both the Earth and Sun, and confirm that the results align with Newton’s Laws and the inverse-square nature of gravitational force.

**Observations**
* After running the simulation, the Earth can be seen following an elliptical pathway, moving farther away from the Sun on the left side of the orbit and remaining closer to the Sun when passing on the right. This behavior indicates that the Earth’s momentum is not constant — it changes direction and magnitude over time as it orbits.
* The Sun’s momentum also changes, although it appears nearly stationary in the visualization. This is due to its enormous mass: the Sun does move, but its velocity is so small compared to Earth’s that its motion is not visually significant. The momentum graphs reveal that the Sun is, in fact, responding to the Earth’s gravitational pull in an equal and opposite fashion, consistent with Newton’s Third Law.

### Graphs
#### Graph #5 - Earth and Sun Momentum Diagram
![Momentum Graph](../assets/img/Lab-2/Run-2/2-2-Momentum.png)

* Red, orange, yellow lines → Sun’s momentum in x, y, z directions.
* Blue, green, black lines → Earth’s momentum in x, y, z directions.
* X-momentum: mirrored between Earth and Sun. When Earth’s x-momentum is positive, the Sun’s is negative, and vice versa. This reflects Newton’s Third Law (equal and opposite forces).
* Y-momentum: the Sun shows steady positive values that increase and then decrease, while Earth’s y-momentum mirrors it in the opposite direction. Again, this symmetry is expected from the mutual gravitational interaction.
* Z-momentum: ≈ 0 for both bodies. This makes sense because no forces are pulling objects into or out of the plane of motion.

#### Graph #6 - Total Momentum Diagram
![Momentum Graph](../assets/img/Lab-2/Run-2/2-2-Total-Momentum.png)

* X-direction: Earth and Sun momenta cancel, resulting in a total near 0 throughout the simulation.
* Y-direction: total momentum remains constant at roughly $2.3 × 10^{29}$ $kg \times m/s$. This confirms momentum conservation—the Earth’s and Sun’s y-components shift in opposite ways, keeping the sum constant.
* Z-direction: remains 0 throughout, consistent with the absence of motion along the z-axis.

#### Graph #7 - Earth and Sun Velocity Diagram
![Velocity Graph](../assets/img/Lab-2/Run-2/2-2-Velocity.png)

* Sun’s velocity is extremely small in x and y, and exactly 0 in z. This matches expectations given its huge mass.
* Earth’s x-velocity: starts at 0, decreases negatively (moving left), then crosses into positive values (moving right). This change reflects its elliptical path and the varying gravitational pull from the Sun.
* Earth’s y-velocity: initially positive but decreases toward 0, eventually becoming negative, then reversing again to positive. This cycle corresponds to the elliptical motion—slowing down as it approaches aphelion (farthest point), then speeding up again as it nears perihelion (closest point).
* These variations align with the inverse-square law of gravity: as the Earth’s distance from the Sun changes, the gravitational pull (and thus its acceleration) changes as well.

#### Graph #8 - Earth and Sun Position Diagram
![Position Graph](../assets/img/Lab-2/Run-2/2-2-Position.png)

* Earth’s y-position: increases, decreases below the axis, then increases again, mirroring the velocity graph. Since velocity is the derivative of position, the correlation is expected.
* Earth’s x-position: shifts from above to below the axis, remaining below by the end of the simulation. This indicates it has not completed a full orbital cycle within the time span observed.
* Sun’s position (especially in z) remains essentially constant, reinforcing that its motion is negligible relative to Earth’s.

#### Conclusion
&emsp;&emsp;This simulation demonstrates that modifying the Earth’s velocity results in an elliptical orbit consistent with Kepler’s First Law. The momentum diagrams confirm Newton’s Third Law by showing equal and opposite momentum shifts between Earth and Sun. The velocity and position graphs align with the inverse-square law of gravity, showing how changes in distance directly affect acceleration and velocity.

&emsp;&emsp;Finally, the absence of a print statement in the output is consistent with the fact that the Earth has not completed a single orbit within the one-year simulation window.

## Simulation #3 – Second Elliptical Orbit
![Simulation](../assets/gifs/Lab-2/Earth_Orbiting_Sun_3.gif)

### Overview
**Setup**: For this simulation, the mass of the Sun was increased to $5 \times 10^{30} kg$. Since the gravitational force is proportional to mass ($F = \frac {G \times m_1 \times m_2} {r^2}$), this change increases the Sun’s pull on the Earth by direct proportion. From momentum ($p = m \times v$), we also expect the Sun’s momentum peaks to be higher, since a larger mass produces greater momentum at the same velocity. The goal is to observe how this adjustment affects the Earth’s orbit, the Sun’s response, and the overall conservation of momentum in the system.

**Observations**: Running the simulation produces an elliptical orbit where the Earth travels farther to the right side of the Sun and remains closer on the left side. As in earlier runs, both Earth and Sun exhibit changing momentum throughout the orbit. The Sun’s motion remains visually small, but the momentum graphs reveal clear oscillations as the Sun responds to Earth’s gravitational pull. These results behave exactly as predicted: stronger gravity produces more tightly bound orbital dynamics and larger momentum magnitudes.

### Graphs
#### Graph #9 - Earth and Sun Momentum Diagram
![Momentum Graph](../assets/img/Lab-2/Run-3/2-3-Momentum.png)

* Sun’s Momentum (red: x, orange: y, yellow: z):
    * In the x-direction, the Sun’s momentum oscillates between positive and negative values, corresponding to it shifting right and left as the Earth pulls on it during orbit.
    * In the y-direction, the Sun’s momentum is mostly positive, showing upward motion with alternating increases and decreases. The momentum occasionally drops to zero when Earth completes a loop, then resumes, reflecting the Sun’s subtle “wobble.”
    * Z-momentum remains 0, as expected.
* Earth’s Momentum (blue: x, green: y, cyan: z):
    * X-momentum mirrors the Sun’s, with equal magnitude but opposite direction—demonstrating Newton’s Third Law.
    * Y-momentum starts positive, decreases to negative, then cycles back. This shows Earth moving upward, slowing, then reversing direction as part of its elliptical trajectory.
    * Z-momentum stays at 0.
* Interpretation: Both Earth and Sun graphs mirror each other, showing equal and opposite exchanges of momentum that align with the simulation’s visual behavior.

#### Graph #10 - Total Momentum Diagram
![Total Momentum Graph](../assets/img/Lab-2/Run-3/2-3-Total-Momentum.png)

* Z-direction: Always 0, since neither body moves along this axis.
* Y-direction (green): Constant at ≈ $1.8 \times 10^29$ $kg \times m/s$. This reflects the sum of Earth and Sun momenta. Individually, they fluctuate in opposite ways, but together their total remains conserved.
* X-direction: Oscillations of Earth and Sun cancel exactly, producing a total near 0.
* Conclusion: The total system momentum is conserved in all directions, consistent with physical law.

#### Graph #11 - Earth and Sun Velocity Diagram
![Velocity Graph](../assets/img/Lab-2/Run-3/2-3-Velocity.png)

* Sun: Velocities in x, y, z are nearly 0, as expected given its massive size. Momentum changes are visible, but the corresponding velocities remain vanishingly small.
* Earth: Velocities follow the same patterns as its momentum, since $p = m \times v$. The graphs confirm that velocity and momentum are directly proportional for Earth, and the paths match exactly what was seen in Simulation #2.

#### Graph #12 - Earth and Sun Position Diagram
![Position Graph](../assets/img/Lab-2/Run-3/2-3-Position.png)

* Earth:
    * X-position begins far right, decreases toward 0, then reverses direction—matching the velocity graph where Earth slows, stops, and accelerates back the other way.
    * Y-position rises positively, then reverses into negative territory, before rising again—mirroring Earth’s y-velocity, which shifts from positive to 0 to negative.
    * Z-position remains at 0, consistent with no z-motion.
* Sun: Position lines are essentially flat due to its minimal velocity. Its “wobble” exists but is too small to be visible on the graph.
* Interpretation: Position graphs behave as expected since velocity is the derivative of position.

#### Conclusion
&emsp;&emsp;By increasing the Sun’s mass, the gravitational force on Earth strengthens, producing more pronounced elliptical motion and larger momentum magnitudes. Momentum graphs confirm equal and opposite interactions between Earth and Sun, while total momentum remains conserved in all directions. Velocity and position diagrams support these findings, showing changes consistent with the gravitational force’s inverse-square relationship with distance.

&emsp;&emsp;The print statement output confirms that Earth completes 16 full orbits during the simulated timespan. This matches the velocity graph, where 16 cycles can be counted from the peaks and dips in the Earth’s motion. Thus, both graphical and numerical outputs align, validating the simulation.

## Simulation #4 - Binary System Run
![Simulation](../assets/gifs/Lab-2/Earth_Orbiting_Sun_4.gif)
### Overview
**Setup**:
* For this simulation, the Earth’s mass was modified to be equivalent to the Sun’s mass ($1.989 \times 10^{30} kg$). This results in a dramatic change in the orbital dynamics: instead of the Earth orbiting the Sun, both celestial bodies orbit around each other.
* From the perspective of gravitational physics, this makes sense. The gravitational force is given by $F = \frac {G \times m_1 \times m_2} {r^2}$, so by increasing the Earth’s mass to equal that of the Sun, the gravitational force increases proportionally. Additionally, since momentum is defined as $p=mv$, the peak momentum values are expected to be greater because the Earth’s increased mass scales its momentum directly.

**Observation**:
* With the the Earth’s mass now equal to 1.989e30 kg, the Earth and Sun now orbit around one another. This means that the Gravitational Force must have increased by direct proportion. Based on the equation, $F = \frac {G \times m_1 \times m_2} {r^2}$, if we increased the mass of a singular object, the Gravitational Force must have increased since the Force is directly proportional to a singular mass’s object. This also means that the peak of the Momentum must be greater, as the Momentum is also directly proportional to a singular object’s mass by the equation $P = mv$.
* Furthermore, the simulation output indicates that the orbital period has decreased to about 139 days per orbit, compared to the 365 day period of the real Earth–Sun system. This result makes sense: by equalizing the Earth and Sun’s masses, the gravitational interaction strengthens, pulling both bodies into faster mutual orbits.

### Graphs

#### Graph #13 - Earth and Sun Momentum Diagram
![Momentum Graph](../assets/img/Lab-2/Run-4/2-4-Momentum.png)

&emsp;&emsp;The momentum components for both the Earth and the Sun show repeating oscillatory patterns of increasing and decreasing values in both the x- and y-directions. This behavior is consistent with the definition of momentum: $p=mv$, where velocity, $v$, is the changing variable. Because the system now completes many more orbits in the same time span, the momentum cycles reflect the constant switching of direction as the two bodies orbit around their common center of mass.
* Sun’s z-momentum (yellow) & Earth’s z-momentum (cyan): These remain constant at 0 kg·m/s, which makes sense since no motion occurs in the z-direction.
* x- and y-momentum (Earth vs. Sun): They mirror each other. When the Earth’s x-momentum is negative, the Sun’s is positive, and vice versa. In the y-direction, both fluctuate around values that don’t perfectly cancel due to the overall upward drift of the system.
The simulation confirms that the gravitational pull is equally effective in both directions, causing the Sun and Earth to loop around each other rather than one body remaining relatively stationary.

#### Graph #14 - Total Momentum Diagram
![Momentum Graph](../assets/img/Lab-2/Run-4/2-4-Total-Momentum.png)

This graph sums the x-, y-, and z-components of the Earth and Sun’s momenta:
* x-momentum (blue): Cancels out to 0 $kg \times m/s$ because Earth and Sun move in opposite directions.
* z-momentum (cyan): Constantly 0 $kg \times m/s$, since no z-motion occurs.
* y-momentum (green): Remains approximately constant at ~$6 \times 10^{34}$ kg·m/s. This arises because the Earth’s y-momentum sometimes dips negative, but the Sun’s y-momentum always remains positive and larger, producing a net positive total.

This result aligns with the physical scenario: while the x-motion cancels, the system as a whole drifts upward in the y-direction.

#### Graph #15 - Earth and Sun Velocity Diagram
![Velocity Graph](../assets/img/Lab-2/Run-4/2-4-Velocity.png)

As expected, the velocity graph mirrors the momentum graph but on a smaller scale (since $p=mv$ and mass is constant).
* x-velocity: Alternates between positive and negative values, showing the oscillatory motion in left–right movement.
* y-velocity: Generally remains positive, with fluctuations into negative values for the Earth, showing the system’s overall upward drift.
* z-velocity: Constant at 0 m/s, consistent with no z-motion.

This confirms that the Sun and Earth move around each other, but also as a system drift upward in the positive y-direction over time.

#### Graph #16 - Earth and Sun Position Diagram
![Position Graph](../assets/img/Lab-2/Run-4/2-4-Position.png)

The position graph reflects the integration of the velocity data:
* x-position: Oscillates within a set range, consistent with left–right looping motion.
* y-position: Shows a repeating pattern but with a steady upward trend, confirming the drift of the entire system.
* z-position: Remains at 0 m, as neither body moves toward or away from the viewer.

Since velocity is the derivative of position, the position curves correctly reflect the slopes shown in the velocity graph.

# Conclusion

At the beginning of this lab, we were tasked with manipulating variables for the Earth–Sun system to generate four different simulations: one circular orbit, two elliptical orbits, and one binary run. Each scenario illustrated the effects of gravitational force and the momentum principle on orbital dynamics.
* Circular Run: Using the given initial parameters, the Earth completed a single circular orbit around the Sun in roughly a year's time.
* Elliptical Run #1: By increasing the Earth’s initial velocity in the y-direction to $1.3 \times 2.978 \times 10^4$ m/s, the Earth followed a much wider orbital path. The increased velocity allowed Earth to resist the Sun’s pull more strongly, increasing orbital radius. This weakened gravitational force ($F∝\frac{1}{r^2}$) and prevented a full orbit within a year.
* Elliptical Run #2: Increasing the Sun’s mass to $5×10^{30}kg$ pulled the Earth into a tighter orbit. This increased gravitational force shortened the orbital period, resulting in 3 orbits per year.
* Binary Run: Equalizing the masses of Earth and Sun caused them to orbit around each other, as their gravitational influence became symmetric. The orbital period decreased to ~139 days, producing 2 full orbits in a year.

**Graphical Analysis**
* The diagrams and print statements provided deeper insights into the system:
    * Momentum Diagrams: Showed how momentum ($p=mv$) varied for both Earth and Sun. For individual bodies, momentum oscillated as direction changed; when summed, the total momentum remained constant, matching physical expectations.
    * Total Momentum Graphs: Confirmed conservation of momentum. While the x- and z-components canceled, the y-component remained a constant nonzero value, reflecting the system’s overall upward drift.
    * Velocity Graphs: Mirrored the momentum graphs on a smaller scale (since $p=mv$). They highlighted speed and directional changes in real time.
    * Position Graphs: Illustrated the orbital paths directly. As expected, velocity acted as the derivative of position, while slopes in the position graph could be used to estimate instantaneous velocity.
    * Print Statements: Counted completed orbits, verifying orbital periods for each simulation (1, <1, 3, and 2 orbits depending on conditions).

**Key Learnings**
Through this lab, we gained both physical and computational insights:
1. Modeling Gravitational Force: Learned how to simulate gravitational interaction between two celestial bodies using radius, force, momentum, and velocity updates.
2. Time Progression: Incorporated time steps ($dt=3600$ s) to track orbital motion over years.
3. Unit Vectors: Applied $\hat{r}$ in the gravitational force equation to calculate directional force.
4. Initial Velocity Calculation: Determined orbital velocity by dividing orbital circumference by period.
5. Momentum Conservation: Observed that the combined momentum of the system remains constant, even as individual bodies oscillate in their momentum.

&emsp;&emsp;In all, this lab demonstrated how relatively small adjustments to initial parameters (velocity, mass) can drastically change orbital mechanics, while also reinforcing fundamental physics principles like conservation of momentum, gravitational force, and the relationship between position, velocity, and time.

## Want to Run it Yourself?
You can interact with this simulation directly in your browser using [GlowScript VPython](https://www.glowscript.org/).
* Click the link below to open the code in GlowScript.
* Press Run to see the simulation.
* No installation is required — everything runs in your browser.
[Run simulation #1 on GlowScript](https://www.glowscript.org/#/user/Tyrese_G/folder/Physics-Labs/program/Lab-2-1)
[Run simulation #2 on GlowScript](https://www.glowscript.org/#/user/Tyrese_G/folder/Physics-Labs/program/Lab-2-2)
[Run simulation #3 on GlowScript](https://www.glowscript.org/#/user/Tyrese_G/folder/Physics-Labs/program/Lab-2-3)
[Run simulation #4 on GlowScript](https://www.glowscript.org/#/user/Tyrese_G/folder/Physics-Labs/program/Lab-2-4)